In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

## Load dataset

In [2]:
df = pd.read_csv("../data/disaster-tweets/train.csv"); df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
# replace urls
import re
url_regex = re.compile("(http|https)://[\w\-]+(\.[\w\-]+)+\S*")

df = df.replace(to_replace=url_regex, value='', regex=True)

## Train a model

In [51]:
clf = make_pipeline(TfidfVectorizer(min_df=10, max_features=10000, stop_words="english"), LogisticRegression())

In [52]:
clf.fit(df.text, df.target)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_features=10000, min_df=10,
                                 stop_words='english')),
                ('logisticregression', LogisticRegression())])

In [53]:
train_pred = clf.predict(df.text)
print(accuracy_score(df.target, train_pred))

0.8418494680152371


In [54]:
parameters = clf.steps[1][1].coef_[0]

In [55]:
parameters

array([-8.08794282e-04,  1.32442554e+00,  6.28500685e-01, ...,
        3.78617130e-01,  8.91140082e-01, -1.38687237e-01])

In [56]:
feature_names = clf.steps[0][1].get_feature_names_out()

In [57]:
feature_names

array(['00', '01', '04', ..., 'ûïwhen', 'ûò', 'ûó'], dtype=object)

In [58]:
# get top words
param_word_map = {p: w for p, w in zip(parameters, feature_names)}

In [59]:
t = sorted(param_word_map.items(), key=lambda x: -np.abs(x[0]))[:100]

In [60]:
t

[(3.8333681092837013, 'hiroshima'),
 (3.0733884975186045, 'california'),
 (3.036781414017624, 'wildfire'),
 (2.8589470573489906, 'killed'),
 (2.619795475448317, 'typhoon'),
 (2.616502339960259, 'bombing'),
 (2.6122298682537965, 'fires'),
 (2.5363012762012387, 'earthquake'),
 (2.5105560717687907, 'buildings'),
 (2.4324469618172757, 'train'),
 (2.4206506546985205, 'suicide'),
 (2.3836688523956764, 'derailment'),
 (2.374864911076289, 'near'),
 (2.334147126383494, 'storm'),
 (2.231100997520618, 'evacuated'),
 (2.205915910274888, 'floods'),
 (2.2007913489727846, 'massacre'),
 (2.179469562564117, 'japan'),
 (2.158615276244241, 'forest'),
 (2.151172188336852, 'police'),
 (2.1424789087605456, 'mh370'),
 (2.116190433196357, 'debris'),
 (2.1118721655047334, 'spill'),
 (2.0830170341569265, 'legionnaires'),
 (2.0686640281729396, 'drought'),
 (2.062780554025578, 'migrants'),
 (2.0255854025373483, 'bridge'),
 (1.9882637904135851, 'atomic'),
 (-1.9753745735105719, 'love'),
 (1.9742247228059298, 'cras

In [61]:
nationalities = ["myanmar", "german", "israel", "japan", "taiwan", "turkish", "pakistan", "english", "finnish", "egypt", "syrian", "palestine",
                 "saudi", "russian", "russia", "syria", "california", "australia", "ukraine", "chinese", "china", "arabia", "greece"]
protected_groups = ["migrants", "woman", "female", "refugee"]
religions = ["jew", "jewish", "muslim", "moslem", "islam", "christian", "christ"]

In [62]:
clf.predict_proba(["I'm an israeli"])

array([[0.23285187, 0.76714813]])

In [63]:
clf.predict_proba(["I'm a muslim."])

array([[0.66956782, 0.33043218]])

In [64]:
clf.predict_proba(["I'm an israeli woman"])

array([[0.37958405, 0.62041595]])

In [65]:
clf.predict_proba(["I'm from japan."])

array([[0.18645087, 0.81354913]])

In [66]:
clf.predict_proba(["We're a family from myanmar."])

array([[0.26384542, 0.73615458]])

In [67]:
clf.predict_proba(["An isreali on a plane"])

array([[0.25579034, 0.74420966]])

In [68]:
clf.predict_proba(["A jew on a plane."])

array([[0.25579034, 0.74420966]])

In [69]:
clf.predict_proba(["A woman on a plane."])

array([[0.40685288, 0.59314712]])

In [80]:
clf.predict_proba(["A man on a plane."])

array([[0.36145207, 0.63854793]])

## Look at clusters

In [70]:
#from bertopic import BERTopic

#topic_model = BERTopic()
#topics, probs = topic_model.fit_transform(df.text)

In [71]:
#topic_model.visualize_documents(df.text)

## run lime on tweets with protected attributes

In [72]:
nationality_tweets = []
for tweet in df.text:
    if any(n in tweet.lower() for n in nationalities):
        nationality_tweets.append(tweet)

In [73]:
len(nationality_tweets)

505

In [74]:
len(nationality_tweets) / len(df.text)

0.06633390253513727

In [75]:
import eli5
from eli5.lime import TextExplainer
from typing import List


def important_lime_words(model, text: str, target_names: List[str], top_n: int = 10):
    te = TextExplainer(random_state=42)
    te.fit(text, model.predict_proba)
    #te.show_prediction(target_names=target_names, feature_names=te.vec_.get_feature_names_out())
    interpret_sample_dict = {c: w for c, w in zip(te.clf_.coef_[0], te.vec_.get_feature_names_out())}
    top_interpret_sample_dict = sorted(interpret_sample_dict.items(), key=lambda x: -np.abs(x[0]))[:min(len(interpret_sample_dict), 10)]
    return [w for (s, w) in top_interpret_sample_dict]

In [76]:
text = nationality_tweets[49]

print(text)
important_lime_words(
    model=clf,
    text=text,
    target_names=["no-disaster", "disaster"]
)

Christian Attacked by Muslims at the Temple Mount after Waving Israeli Flag via Pamela Geller - ... 


/home/tobias/.cache/pypoetry/virtualenvs/biaslyze-OjqpgMw8-py3.8/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:165: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


['israeli',
 'temple',
 'mount',
 'muslims',
 'christian',
 'waving',
 'geller',
 'waving israeli',
 'flag',
 'temple mount']

In [82]:
important_lime_words(
    model=clf,
    text="I am an isreali man",
    target_names=["no-disaster", "disaster"]
)

/home/tobias/.cache/pypoetry/virtualenvs/biaslyze-OjqpgMw8-py3.8/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:165: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


['man',
 'isreali',
 'am',
 'an',
 'i',
 'an isreali',
 'am an',
 'isreali man',
 'i am']

## does changing the protected attribute change the prediction?

In [77]:
text = nationality_tweets[63]
print(text)

clf.predict_proba([text])

I liked a @YouTube video  FEMA REGION III TARGETED for BIOTERRORISM !!! NASA - JAPAN ROCKET LAUNCH with LITHIUM


array([[0.34110311, 0.65889689]])

In [78]:
text2 = text.replace("German", "Muslim")
print(text2)

clf.predict_proba([text2])

I liked a @YouTube video  FEMA REGION III TARGETED for BIOTERRORISM !!! NASA - JAPAN ROCKET LAUNCH with LITHIUM


array([[0.34110311, 0.65889689]])

In [79]:
important_lime_words(
    model=clf,
    text=text,
    target_names=["no-disaster", "disaster"]
)

/home/tobias/.cache/pypoetry/virtualenvs/biaslyze-OjqpgMw8-py3.8/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:165: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


['japan',
 'region',
 'bioterrorism',
 'liked',
 'youtube',
 'video',
 'youtube video',
 'video fema',
 'lithium',
 'launch']